![ecommerce_analytics-1224x532](ecommerce_analytics-1224x532.jpg)


As a Data Engineer at an electronics e-commerce company, Voltmart, you have been requested by a peer Machine Learning team to clean the data containing the information about orders made last year. They are planning to further use this cleaned data to build a demand forecasting model. To achieve this, they have shared their requirements regarding the desired output table format.

An analyst shared a parquet file called `"orders_data.parquet"` for you to clean and preprocess. 

You can see the dataset schema below along with the **cleaning requirements**:

## `orders_data.parquet`

| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `order_date` | `timestamp` | Date and time when the order was made | _Modify: Remove orders placed between 12am and 5am (inclusive); convert from timestamp to date_ |
| `time_of_day` | `string` | Period of the day when the order was made | _New column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm_ |
| `order_id` | `long` | Order ID | _N/A_ |
| `product` | `string` | Name of a product ordered | _Remove rows containing "TV" as the company has stopped selling this product; ensure all values are lowercase_ |
| `product_ean` | `double` | Product ID | _N/A_ |
| `category` | `string` | Broader category of a product | _Ensure all values are lowercase_ |
| `purchase_address` | `string` | Address line where the order was made ("House Street, City, State Zipcode") | _N/A_ |
| `purchase_state` | `string` | US State of the purchase address | _New column containing: the State that the purchase was ordered from_ |
| `quantity_ordered` | `long` | Number of product units ordered | _N/A_ |
| `price_each` | `double` | Price of a product unit | _N/A_ |
| `cost_price` | `double` | Cost of production per product unit | _N/A_ |
| `turnover` | `double` | Total amount paid for a product (quantity x price) | _N/A_ |
| `margin` | `double` | Profit made by selling a product (turnover - cost) | _N/A_ |

<br>

In [64]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

In [65]:
orders_data = spark.read.parquet('orders_data.parquet')
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


In [66]:
# Start here, using as many cells as you require
orders_data=orders_data.filter(
    ~((F.hour('order_date')>= 0) & (F.hour('order_date')<= 5) )
)

# Convert to Pandas DataFrame and display the head
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


In [67]:
# Add 'time_of_day' column based on the hour of 'order_date'
orders_data = orders_data.withColumn(
    'time_of_day',
    F.when((F.hour('order_date') >= 5) & (F.hour('order_date') < 12), 'morning')
     .when((F.hour('order_date') >= 12) & (F.hour('order_date') < 18), 'afternoon')
     .when((F.hour('order_date') >= 18) & (F.hour('order_date') < 24), 'evening')
     .otherwise('night')
)





In [68]:
# Convert 'order_date' from timestamp to date
orders_data = orders_data.withColumn(
    'order_date', orders_data['order_date'].cast('date')
)



In [69]:
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day
0,2023-01-22,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,evening
1,2023-01-28,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,afternoon
2,2023-01-17,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,afternoon
3,2023-01-05,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,evening
4,2023-01-25,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,morning


In [70]:
# Ensure all values in 'product' are lowercase
orders_data = orders_data.withColumn(
    'product', F.lower(F.col('product'))
)

In [71]:
orders_data=orders_data.filter(
    ~F.col('product').contains('tv')
  
    
)

In [72]:
# Ensure all values in 'category' are lowercase
orders_data=orders_data.withColumn(
'category',F.lower(F.col('category'))
)



In [73]:
# Extract the state abbreviation from the address
orders_data = orders_data.withColumn('purchase_state', F.regexp_extract(F.col('purchase_address'), r', (\w{2}) \d{5}$', 1))

# Convert to Pandas DataFrame and display the head
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day,purchase_state
0,2023-01-22,141234,iphone,5.638009e+12,vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,evening,MA
1,2023-01-28,141235,lightning charging cable,5.563320e+12,alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,afternoon,OR
2,2023-01-17,141236,wired headphones,2.113973e+12,vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,afternoon,CA
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,evening,CA
4,2023-01-25,141238,wired headphones,9.692681e+12,électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,morning,TX


In [74]:
# Repartition the DataFrame by 'order_date' into 8 partitions
orders_data_repartitioned = orders_data.repartition(8, 'order_date')

# Save the DataFrame as a Parquet file
orders_data_repartitioned.write.mode('overwrite').parquet('orders_data_clean.parquet')